<a href="https://colab.research.google.com/github/kevincong95/cs231n-emotiw/blob/master/notebooks/2.0-la-tj-ak-ensemble_baseline_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Video Sentiment Analysis in the Wild
### Ensembling Notebook | CS231n

This notebook preprocesses input videos to extract faces, frames, poses, and audio before running pre-trained models for each modality to predict group sentiment (positive, negative, or neutral). 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Clone the code base
!git clone 'https://github.com/kevincong95/cs231n-emotiw.git'

# Switch to TF 1.x and navigate to the directory
%tensorflow_version 1.x

%cd cs231n-emotiw
!pip install -r 'requirements-predictions.txt'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Cloning into 'cs231n-emotiw'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 903 (delta 132), reused 105 (delta 47), pack-reused 683
Receiving objects: 100% (903/903), 187.00 MiB | 36.64 MiB/s, done.
Resolving deltas: 100% (540/540), done.
TensorFlow 1.x selected.
/content/cs231n-emotiw
     |████████████████████████████████| 389kB 10.9MB/s 
     |██████

#### Pose Pre-Requisites
Pose extraction uses the [CMU OpenPose library](https://github.com/CMU-Perceptual-Computing-Lab/openpose) to extract body keypoints. We have pre-compiled this library for use in Colab but some system files still need to be installed. 

#### Retrieve the files

The code block below demonstrates how to retrieve the files from GCS. However, feel free to skip this step if the files are already on the local disk or you have Google Drive mounted.

In [0]:
!wget -q https://storage.googleapis.com/cs231n-emotiw/data/train-full.zip
!wget -q https://storage.googleapis.com/cs231n-emotiw/data/val-full.zip

!unzip -q "train-full.zip"
!unzip -q "val-full.zip"

In [3]:
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
!wget https://storage.googleapis.com/cs231n-emotiw/openpose/openpose.tar.gz
!tar -xzf openpose.tar.gz

# The pre-built OpenPose library contains shared library files that need to be manually linked
import os
orig_lib_path = os.environ['LD_LIBRARY_PATH']
os.environ["LD_LIBRARY_PATH"] = f"{orig_lib_path}:{os.getcwd()}/openpose/build/src/openpose/:{os.getcwd()}/openpose/build/caffe/lib/"

Selecting previously unselected package libgflags2.2.
(Reading database ... 144467 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

#### Preprocess Files

Here, we will instantiate each of the preprocessors and process all of the input video files.

NOTE: Change the input parameters as needed.

WARNING: This may take several hours to complete, depending on the number of files.

In general, pre-processing will extract the following:
- Video frames
- Pose keypoints
- Faces from each video frame
- Audio waveform and audio features

In [0]:
!wget -q https://storage.googleapis.com/cs231n-emotiw/data/Train_labels.txt
!wget -q https://storage.googleapis.com/cs231n-emotiw/data/Val_labels.txt

In [0]:
%load_ext autoreload
%autoreload 2

In [6]:
%tensorflow_version 1.x

%cd cs231n-emotiw

[Errno 2] No such file or directory: 'cs231n-emotiw'
/content/cs231n-emotiw


In [0]:
from src.preprocessors.preprocess_all_modes import preprocess
from src.preprocessors.pose_preprocessor import PosePreprocessor

# print("Starting to preprocess train data")
# preprocess(video_folder="train-tiny.zip", label_file="Train_labels.txt", local_base_path="train-tiny")

# print("Starting to preprocess val data")
# preprocess(video_folder="val-tiny.zip", label_file="Val_labels.txt", local_base_path="val-tiny")

# print("Starting to preprocess test data")
# preprocess(video_folder="test-tiny.zip", local_base_path="test-tiny")

print("Starting to preprocess train data")
preprocess(video_folder="Train", label_file="Train_labels.txt", local_base_path="train-full")

!cp train-full-*.zip "/content/drive/My Drive/cs231n-project/datasets/emotiw"

print("Starting to preprocess val data")
preprocess(video_folder="Val", label_file="Val_labels.txt", local_base_path="val-full")

!cp val-full-*.zip "/content/drive/My Drive/cs231n-project/datasets/emotiw"

Using TensorFlow backend.


Starting to preprocess train data
Video Preprocessor created with video_folder = Train , label_file = Train_labels.txt , output_folder = train-full-frames, output_file = train-full-frames.zip
Frames will be created with height = 320 , width = 480 , sample_every = 10
Video Preprocessor created with video_folder = Train , output_folder = train-full-faces, output_file = train-full-faces.zip
Frames will be created with height = 320 , width = 480 , sample_every = 10
FER Preprocessor created with faces_folder = train-full-faces, output_folder = train-full-fer, output_file = train-full-fer.zip
Pose Preprocessor created with is_test = False, video_frame_folder = train-full-frames , output_folder = train-full-pose, output_file = train-full-pose.zip
Video Preprocessor created with video_folder = Train , output_folder = train-full-audio, output_file = train-full-audio.zip
Frames will be created with hop_size = 0.5
Skipping unzipping files as input is a folder
Found 2661 videos
Processing video 8/

In [0]:
!ls train-full-frames_tmp

In [0]:
# Remove the openpose folder as it is no longer required
!rm -rf openpose/

In [0]:
!cp train-full-*.zip "/content/drive/My Drive/cs231n-project/datasets/emotiw/6-4"
!cp val-full-*.zip "/content/drive/My Drive/cs231n-project/datasets/emotiw/6-4"
# !cp test-full-*.zip "/content/drive/My Drive/cs231n-project/datasets/emotiw"